In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv("../input/breast-cancer-wisconsin-benign-or-malignant/tumor.csv")
data.Class = [1 if each == 4 else 0 for each in data.Class]

# Select x and y values
x_not_normalize= data.drop(["Class","Sample code number"],axis = 1)
y = data.Class.values

# Normalizing x values
x = (x_not_normalize +1 - np.min(x_not_normalize))/ (np.max(x_not_normalize) - np.min(x_not_normalize)).values

# Create train and test data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

# Transpoze all data
x_test = x_test.T
x_train = x_train.T
y_test = y_test.T
y_train = y_train.T

In [ ]:
# initialize weight and bias
def initialize_weight_and_bias(dimention):
    w = np.full((dimention,1), 0.01)
    b = 0.0
    return w,b

In [ ]:
# sigmoid function
def sigmoid(z):
    y_head = 1 / (1 + np.exp(-z))
    return y_head

In [ ]:
# forward and backward propagation
def forward_backward_propagation(w,b,x_train,y_train):
    #forward propagation
    z = np.dot(w.T, x_train)+b
    y_head = sigmoid(z)
    loss = -(1-y_train)*np.log(1-y_head) - (y_train*np.log(y_head))
    cost = np.sum(loss) / x_test.shape[1]   
    #backward propagation
    derivative_weight = (np.dot(x_train, (y_head - y_train).T)) / x_train.shape[1]
    derivative_bias = np.sum(y_head-y_train)/x_train.shape[1]
    gradients = {"derivative_weight":derivative_weight, "derivative_bias":derivative_bias}
    
    return cost, gradients

In [ ]:
def update(w,b,x_train, y_train, learning_rate, number_of_iteration):
    cost_list = []
    cost_list2 = []
    index = []
    
    for i in range(number_of_iteration):
        cost,gradients = forward_backward_propagation(w,b,x_train, y_train)
        
        cost_list.append(cost)
        
        w = w - learning_rate * gradients["derivative_weight"]
        b = b - learning_rate * gradients["derivative_bias"]
        
        
        if i%10 == 0:
            cost_list2.append(cost)
            index.append(i)
            print("Cost after iteration {}: {}".format(i,cost))

    parameters = {"weight":w, "bias":b}
    plt.plot(index,cost_list2)
    plt.xticks(index,rotation="vertical")
    plt.xlabel("Number Of Iteration")
    plt.ylabel("Cost")
    plt.show()
    
    return parameters, gradients, cost_list

In [ ]:
def predict(w,b,x_test):
    Y_head = sigmoid(np.dot(w.T,x_test)+b)
    Y_predicted = np.zeros((1,x_test.shape[1]))
    
    for i in range(Y_head.shape[1]):
        if Y_head[0,i] <= 0.5:
            Y_predicted[0,i] = 0
        else:
            Y_predicted[0,i] = 1
    return Y_predicted

In [ ]:
def logistic_regression(x_train, y_train, x_test, y_test, learning_rate, num_iteration):
    w,b = initialize_weight_and_bias(dimention = x_train.shape[0])
    
    parameters, gradients, cost_list = update(w,b,x_train,y_train, learning_rate, num_iteration)
    
    y_predicted = predict(parameters["weight"],parameters["bias"], x_test)
    
    print("Test accuracy : {}".format(100 - np.mean(np.abs(y_predicted - y_test))*100))
    return y_predicted

In [ ]:
y_predicted = logistic_regression(x_train, y_train, x_test, y_test, learning_rate = 6, num_iteration = 500)

error_count=0
total = 0
# compare the predicted results with real results
for i,j in enumerate(y_predicted[0]):
    if j != y_test[i]:
        error_count += 1
    total+=1
print("{} wrong predict from {} feature.".format(total,error_count))